In [1]:
import tensorflow as tf

D:\Python3.6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pooling_2X2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W)+b)

def fully_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

In [14]:
import numpy as np
from keras.utils import to_categorical

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[:1000]
x_test = x_test[:100]
y_train = y_train[:1000]
y_test = y_test[:100]

# print(x_train.shape)

# x_train = np.resize(x_train, [1000, 784])
# x_test = np.resize(x_test, [100, 784])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train.shape)
print(y_test.shape)
print(y_train.dtype)
print(y_test.dtype)

(1000, 10)
(100, 10)
float64
float64


In [4]:
X = tf.placeholder(tf.float32, shape=[None, 28, 28])
y = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.expand_dims(X, 3) # ?,28,28,1
conv1 = conv_layer(x_image, [5,5,1,32])
conv1_pool = max_pooling_2X2(conv1)

conv2 = conv_layer(conv1_pool, [5,5,32,64])
conv2_pool = max_pooling_2X2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1,7*7*64])
full_1 = tf.nn.relu(fully_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

full_2 = fully_layer(full1_drop, 10)

In [5]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=full_2)
gd = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(full_2, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [15]:
STEPS = 100
MINIBATCH_SIZE = 100
init = tf.global_variables_initializer()

split_X = np.split(x_train, int(x_train.shape[0]/MINIBATCH_SIZE))
split_y = np.split(y_train, int(y_train.shape[0]/MINIBATCH_SIZE))

n_test = int(x_test.shape[0]/MINIBATCH_SIZE)
split_X2 = np.split(x_test, int(x_test.shape[0]/MINIBATCH_SIZE))
split_y2 = np.split(y_test, int(y_test.shape[0]/MINIBATCH_SIZE))

with tf.Session() as sess:
    sess.run(init)    
    for i in range(STEPS):
        k = 0
        if i % 10 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={X:split_X[k], y:split_y[k], keep_prob:1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))
        sess.run(gd, feed_dict={X:split_X[k], y:split_y[k], keep_prob:0.5})
        k += 1
    test_accuracy = np.mean([sess.run(accuracy, feed_dict={X:split_X2[i], y:split_y2[i], keep_prob:1.0}) for i in range(n_test)])
    print("test accuracy: {}".format(test_accuracy))

step 0, training accuracy 0.07000000029802322
step 10, training accuracy 0.3400000035762787
step 20, training accuracy 0.6100000143051147
step 30, training accuracy 0.8899999856948853
step 40, training accuracy 0.9700000286102295
step 50, training accuracy 0.9599999785423279
step 60, training accuracy 0.9900000095367432
step 70, training accuracy 1.0
step 80, training accuracy 1.0
step 90, training accuracy 1.0
test accuracy: 0.7300000190734863
